# Skin Cancer Classification with ISIC 📓


Este notebook demonstra um *pipeline* completo para treinar um classificador de melanoma
usando o **ISIC Archive** e os metadados já estratificados em `folds_13062020.csv`.

### Conteúdo
1. Instalação de dependências  
2. Carregamento do CSV e inspeção inicial  
3. Download das imagens via API ISIC  
4. *Dataset* PyTorch + **Albumentations**  
5. Treino com 5 *folds* (EfficientNet‑B3)  
6. Avaliação (ROC‑AUC)  
7. Exportação (`TorchScript`) e inferência de exemplo  


In [2]:
import pandas as pd, numpy as np, torch, timm, requests, time, os
from pathlib import Path
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm


/home/giovanni/Documents/git/medicine/dermalert/dermalert-ml/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CSV_PATH = 'metadata/folds_13062020.csv'
df = pd.read_csv(CSV_PATH)
df['image_id'] = ["_".join(x.split('_')[:-1 if len(x.split('_')) > 2 else 2]) for x in df['image_id']]
print(df.head())
print("\nTarget distribution (0=benign, 1=melanoma):")
print(df['target'].value_counts())
NUM_FOLDS = df['fold'].nunique()


       image_id  patient_id  target  source     sex  age_approx  \
0  ISIC_2637011  IP_7279968       0  ISIC20    male        45.0   
1  ISIC_0015719  IP_3075186       0  ISIC20  female        45.0   
2  ISIC_0052212  IP_2842074       0  ISIC20  female        50.0   
3  ISIC_0068279  IP_6890425       0  ISIC20  female        45.0   
4  ISIC_0074268  IP_8723313       0  ISIC20  female        55.0   

  anatom_site_general_challenge  stratify_group  fold  
0                     head/neck              31     0  
1               upper extremity               7     2  
2               lower extremity               5     4  
3                     head/neck               7     0  
4               upper extremity               6     4  

Target distribution (0=benign, 1=melanoma):
target
0    52302
1     4922
Name: count, dtype: int64


In [9]:
API_URL = "https://api.isic-archive.com/api/v2/images/{id}/"
CACHE_DIR = Path('images')
CACHE_DIR.mkdir(exist_ok=True)

def fetch_image(isic_id: str) -> Path:
    """Baixa a imagem via API ISIC (se ainda não estiver em cache)."""
    dest = CACHE_DIR / f"{isic_id}.jpg"
    if dest.exists():
        return dest
    for _ in range(3):  # 3 tentativas
        r = requests.get(API_URL.format(id=isic_id), timeout=30)
        if r.ok:
            path_download = r.json()['files']['full']['url']
            r = requests.get(path_download, timeout=30)
            dest.write_bytes(r.content)
            return dest
        time.sleep(2)
    raise RuntimeError(f'Falha ao baixar {isic_id}')

# Total de imagens ja baixadas:
print("Total de imagens já baixadas:", len(list(CACHE_DIR.glob('*'))))


Total de imagens já baixadas: 1543


In [5]:
mean, std = [0.5]*3, [0.5]*3
train_tfms = A.Compose([
    A.LongestMaxSize(512),
    A.PadIfNeeded(512, 512),
    A.RandomRotate90(),
    A.OneOf([
        A.HorizontalFlip(p=1),
        A.VerticalFlip(p=1),
    ], p=0.5),
    A.RandomBrightnessContrast(0.2, 0.2),
    A.ShiftScaleRotate(shift_limit=0.05,
                       scale_limit=0.1,
                       rotate_limit=20,
                       border_mode=0),
    A.Normalize(mean, std),
    ToTensorV2(),
])
val_tfms = A.Compose([
    A.LongestMaxSize(512), A.PadIfNeeded(512, 512),
    A.Normalize(mean, std), ToTensorV2()
])


/home/giovanni/Documents/git/medicine/dermalert/dermalert-ml/.venv/lib/python3.13/site-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [6]:
class ISICDataset(Dataset):
    def __init__(self, df, transforms):
        import numpy as np
        self.df = df.reset_index(drop=True)
        self.transforms = transforms
        self.np = np
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = fetch_image(row.image_id)
        img = Image.open(img_path).convert('RGB')
        img = self.transforms(image=self.np.array(img))['image']
        label = torch.tensor(row.target).long()
        return img, label


In [7]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    losses = []
    for x, y in tqdm(dataloader, leave=False):
        x, y = x.to(device), y.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return sum(losses) / len(losses)

def evaluate(model, dataloader, device):
    model.eval()
    preds, gts = [], []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            logits = model(x)
            preds.extend(torch.sigmoid(logits).cpu().numpy().ravel())
            gts.extend(y.numpy())
    return roc_auc_score(gts, preds)


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
EPOCHS = 10
BATCH = 16
scores = []

for fold in range(NUM_FOLDS):
    print(f"\n===== Fold {fold}/{NUM_FOLDS-1} =====")
    tr_df = df[df.fold != fold]
    vl_df = df[df.fold == fold]
    
    train_ds = ISICDataset(tr_df, train_tfms)
    val_ds   = ISICDataset(vl_df, val_tfms)
    dl_tr = DataLoader(train_ds, BATCH, shuffle=True, num_workers=os.cpu_count()//2 or 2)
    dl_vl = DataLoader(val_ds, BATCH*2, shuffle=False, num_workers=os.cpu_count()//2 or 2)
    
    model = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=1).to(device)
    
    pos_weight = (tr_df.target==0).sum() / (tr_df.target==1).sum()
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight], device=device))
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    
    best_auc, best_state = 0, None
    for epoch in range(EPOCHS):
        loss = train_one_epoch(model, dl_tr, optimizer, criterion, device)
        auc  = evaluate(model, dl_vl, device)
        print(f"Epoch {epoch+1}/{EPOCHS} – loss {loss:.4f} – val AUC {auc:.4f}")
        if auc > best_auc:
            best_auc, best_state = auc, model.state_dict()
    
    torch.save(best_state, f"model_fold{fold}.pt")
    scores.append(best_auc)
    print(f"Fold {fold} best AUC: {best_auc:.4f}")
    
print("\nMédia AUC dos folds:", np.mean(scores))


===== Fold 0/4 =====


/home/giovanni/Documents/git/medicine/dermalert/dermalert-ml/.venv/lib/python3.13/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name efficientnet_b3a to current efficientnet_b3.
  model = create_fn(


KeyboardInterrupt: 

In [ ]:
# Carrega o melhor modelo (ajuste o fold se quiser outro)
model = timm.create_model('efficientnet_b3a', pretrained=False, num_classes=1)
model.load_state_dict(torch.load('model_fold0.pt', map_location='cpu'))
model.eval()
scripted = torch.jit.script(model)
scripted.save('skin_risk_classifier.pt')
print("Modelo exportado em skin_risk_classifier.pt")


In [ ]:
# Demonstração de inferência
sample_id = df.iloc[0].image_id
img_path = fetch_image(sample_id)
from IPython.display import display
display(Image.open(img_path).resize((256,256)))

proc = val_tfms(image=np.array(Image.open(img_path)))['image'].unsqueeze(0)
with torch.no_grad():
    prob = torch.sigmoid(model(proc))[0,0].item()
print(f"Probabilidade de melanoma para {sample_id}: {prob:.4f}")
